# Import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

from neuralart.data import get_chan_data, create_dataset

# Download Wikiart Data from Chan repository

```bash
wget http://web.fsktm.um.edu.my/~cschan/source/ICIP2017/wikiart.zip
````



```bash
wget http://web.fsktm.um.edu.my/~cschan/source/ICIP2017/wikiart_csv.zip
```

# Parse Chan's dataset

In [3]:
# Path to the folder containing Chan's csv files (ie. wikiart_csv)
chan_csv_folder_path = "../raw_data/wikiart/csv_chan"
# Path to the folder containing Chan's images from wikiart (ie. wikiart)
chan_image_folder_path = "../raw_data/wikiart/dataset_chan"
# Path to save a new csv file containing all the information about Chan's dataset
csv_output_path = "../raw_data/wikiart/new_dataset"

chan_data = get_chan_data(chan_csv_folder_path, 
                          chan_image_folder_path, 
                          csv_output_path=csv_output_path)
chan_data.shape

(78748, 10)

# Create a New Dataset

In [4]:
# Dictionary used to merge or drop some classes
merge={'name': 'style_m1',
       'merging':{'abstract_expressionism': 'abstract', 'action_painting': 'abstract', 
                  'analytical_cubism': 'cubism', 'art_nouveau_modern': None, 'baroque': None, 
                  'color_field_painting': 'color_field_painting', 'contemporary_realism': None, 
                  'cubism': 'cubism', 'early_renaissance': 'renaissance', 
                  'expressionism': 'expressionism', 'fauvism': None, 'high_renaissance': 'renaissance', 
                  'impressionism': 'impressionism', 'mannerism_late_renaissance': None, 
                  'minimalism': None, 'naive_art_primitivism': None, 'new_realism': None, 
                  'northern_renaissance': 'renaissance', 'pointillism': None, 'pop_art': None,
                  'post_impressionism': None, 'realism': 'realism', 'rococo': None, 
                  'romanticism': 'romanticism', 'symbolism': None, 'synthetic_cubism': 'cubism', 
                  'ukiyo_e': None}}

# Path to save a new csv file containing all the information about the new dataset
csv_file_name =  "../raw_data/wikiart/test/wikiart-target_style-class_27.csv"
# Path to the folder containing Chan's images from wikiart (ie. wikiart)
chan_image_folder_path = "../raw_data/wikiart/dataset_chan"
# Path to create a new directory containing all the wikiart images used in the new dataset
image_folder_output_path = "../raw_data/wikiart/new_dataset"

# Train, Val, and test ratio to split the new dataset
val_ratio=0.1
test_ratio=0.1

# Sampling wikiart dataset using n images per class
n=100
strategy="max"

data=create_dataset(csv_file_name, 
                    merge=merge, 
                    n=n, 
                    strategy=strategy, 
                    random_state=123,
                    chan_image_folder_path=chan_image_folder_path, 
                    csv_output_path=csv_output_path, 
                    image_folder_output_path=image_folder_output_path,
                    val_ratio=val_ratio,
                    test_ratio=test_ratio)

Done: 800 image(s) copied
